In [2]:
# Global vars
task = "ner"
model_checkpoint = "distilbert-base-uncased"
batch_size = 32
seed = 0

In [68]:
from datasets import load_dataset, ClassLabel, Sequence

# Loading training file
dataset = load_dataset('json', data_files='../input/train_ner.json', features={'train': ClassLabel(num_classes=3, names=['B', 'O', 'I'])})

# Train valid split
datasets = dataset['train'].train_test_split(test_size=0.05, seed=0)

# Show characteristics
datasets

Using custom data configuration default-a57be1678b8d42d9


TypeError: argument of type 'ClassLabel' is not iterable

In [60]:
datasets["train"].features['token']

Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)

In [64]:
type(datasets["train"].features['token'].feature)

datasets.features.Value

In [25]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [26]:
show_random_elements(datasets["train"])

,token,tags
0,"[receive, cCRT, in, accordance, with, standard, of, care, guidelines, Other, studies, have, demonstrated, improved, quality, and, timeliness, of, care, for, NSCLC, patients, who, receive, multidisciplinary, management, and, who, are, referred, to, multiple, types, of, cancer, specialists, 18, 27, This, study, found, that, the, proportion, of, elderly, patients, receiving, cCRT, over, single, modality, therapy, increased, in, the, years, after, 2009, indicative, of, greater, adoption, of]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,"[are, referred, to, as, ADNI, 1, ADNI, 2, and, ADNI, GO, The, fourth, dataset, comes, from, the, Australian, Imaging, Biomarker, and, Lifestyle, Flagship, Study, of, Aging, AIBL, 1, For, ADNI, studies, a, subset, of, participants, was, selected, such, that, there, was, no, overlap, of, participants, amongst, datasets, The, imaging, feature, space, consisted, of, 78, features, as, derived, from, 78, regions, of, interest, ROIs, based]","[O, O, O, O, B, O, B, O, O, B, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
2,"[of, major, disciplines, and, interdisciplinary, fields, spanning, the, physical, and, life, sciences, mathematics, behavioral, and, social, sciences, engineering, and, humanities, They, were, awarded, to, individuals, in, research, based, fields, of, study, who, demonstrated, superior, scholarship, and, showed, greatest, promise, for, future, achievements, as, scholars, researchers, and, teachers, in, institutions, of, higher, education, as, determined, through, a, rigorous, review, process, conducted, by, leading, scholars, in]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
3,"[system, data, layers, national, Census, of, Agriculture, from, the, U, S, Department, of, Agriculture, USDA, National, Agricultural, Statistics, Service, NASS, https, www, agcensus, usda, gov, so, data, filling, must, be, used, to, estimate, data, for, the, years, between, the, years, the, data, are, reported, Another, weak, point, in, existing, estimation, methods, is, reliance, on, old, reports, to, estimate, factors, such, as, agricultural, incidental, use]","[O, O, O, O, B, I, I, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
4,"[in, summer, 1992, Average, amount, includes, only, students, with, loans, 5Auendanec, adjusted, total, cost, minus, total, aid, Attendance, adjusted, budget, minus, EFC, and, aid, Not, applicable, NOTE, Totals, include, students, with, missing, data, on, characteristics, shown, in, the, detail, Therel, ore, the, percentages, or, amounts, for, all, students, may, be, higher, or, lower, than, any, of, the, percentages, or, amounts, shown, in, the, detail]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"


In [27]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading: 100%|██████████| 442/442 [00:00<00:00, 111kB/s]
Downloading: 100%|██████████| 232k/232k [00:00<00:00, 786kB/s]
Downloading: 100%|██████████| 466k/466k [00:00<00:00, 1.23MB/s]
Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 9.33kB/s]


In [28]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [29]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 7592, 1010, 2023, 2003, 2028, 6251, 999, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [31]:
example = datasets["train"][4]
print(example["token"])

['examination', 'of', 'data', 'from', 'the', '2000', 'Census', 'illustrates', 'this', 'pattern', 'in', 'U', 'S', 'urban', 'areas', 'Figure', '5', 'Figure', '5', 'shows', 'the', 'relationship', 'between', 'racial', 'Black', 'White', 'segregation', 'and', 'within', 'race', 'income', 'segregation', 'in', 'the', '228', 'metropolitan', 'area', 'central', 'cities', 'with', 'at', 'least', '5', 'Black', 'population', 'in', '2000', 'Although', 'racial', 'segregation', 'is', 'high', 'in', 'many', 'of', 'these', 'cities', 'with', 'an', 'average', 'of', '0', '31', 'recall']


In [32]:
tokenized_input = tokenizer(example["token"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'examination', 'of', 'data', 'from', 'the', '2000', 'census', 'illustrates', 'this', 'pattern', 'in', 'u', 's', 'urban', 'areas', 'figure', '5', 'figure', '5', 'shows', 'the', 'relationship', 'between', 'racial', 'black', 'white', 'segregation', 'and', 'within', 'race', 'income', 'segregation', 'in', 'the', '228', 'metropolitan', 'area', 'central', 'cities', 'with', 'at', 'least', '5', 'black', 'population', 'in', '2000', 'although', 'racial', 'segregation', 'is', 'high', 'in', 'many', 'of', 'these', 'cities', 'with', 'an', 'average', 'of', '0', '31', 'recall', '[SEP]']


In [1]:
tokenized_input

NameError: name 'tokenized_input' is not defined

In [34]:
len(example["tags"]), len(tokenized_input["input_ids"])

(64, 66)

In [36]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, None]


In [37]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example["tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

66 66


In [39]:
label_all_tokens = True

In [40]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["token"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [45]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/596 [00:00<?, ?ba/s]


ArrowInvalid: Could not convert 'O' with type str: tried to convert to int